1.

In [1]:
import random
import string
import secrets
from typing import List, Optional

class CustomRandomGenerator:
    """
    A class for generating random strings using custom subsets of digits and alphabets.
    """
    
    def __init__(self, 
                 digits: Optional[str] = None,
                 lowercase: Optional[str] = None, 
                 uppercase: Optional[str] = None,
                 special_chars: Optional[str] = None):
        """
        Initialize the generator with custom character subsets.
        
        Args:
            digits: Subset of digits to use (default: '0123456789')
            lowercase: Subset of lowercase letters (default: 'abcdefghijklmnopqrstuvwxyz')
            uppercase: Subset of uppercase letters (default: 'ABCDEFGHIJKLMNOPQRSTUVWXYZ')
            special_chars: Special characters to include (default: None)
        """
        self.digits = digits or string.digits
        self.lowercase = lowercase or string.ascii_lowercase
        self.uppercase = uppercase or string.ascii_uppercase
        self.special_chars = special_chars or ""
        
        # Combine all character sets
        self.charset = self.digits + self.lowercase + self.uppercase + self.special_chars
    
    def generate_random_string(self, length: int, secure: bool = False) -> str:
        """
        Generate a random string of specified length.
        
        Args:
            length: Length of the string to generate
            secure: Use cryptographically secure random (default: False)
            
        Returns:
            Random string of specified length
        """
        if secure:
            return ''.join(secrets.choice(self.charset) for _ in range(length))
        else:
            return ''.join(random.choice(self.charset) for _ in range(length))
    
    def generate_with_constraints(self, 
                                length: int,
                                min_digits: int = 0,
                                min_lowercase: int = 0,
                                min_uppercase: int = 0,
                                min_special: int = 0,
                                secure: bool = False) -> str:
        """
        Generate a random string with minimum character type constraints.
        
        Args:
            length: Total length of the string
            min_digits: Minimum number of digits required
            min_lowercase: Minimum number of lowercase letters required
            min_uppercase: Minimum number of uppercase letters required
            min_special: Minimum number of special characters required
            secure: Use cryptographically secure random
            
        Returns:
            Random string meeting the constraints
        """
        if min_digits + min_lowercase + min_uppercase + min_special > length:
            raise ValueError("Sum of minimum requirements exceeds total length")
        
        result = []
        rand_func = secrets.choice if secure else random.choice
        
        # Add required characters
        result.extend(rand_func(self.digits) for _ in range(min_digits))
        result.extend(rand_func(self.lowercase) for _ in range(min_lowercase))
        result.extend(rand_func(self.uppercase) for _ in range(min_uppercase))
        result.extend(rand_func(self.special_chars) for _ in range(min_special))
        
        # Fill remaining positions
        remaining = length - len(result)
        result.extend(rand_func(self.charset) for _ in range(remaining))
        
        # Shuffle the result
        if secure:
            # Secure shuffle using secrets
            for i in range(len(result) - 1, 0, -1):
                j = secrets.randbelow(i + 1)
                result[i], result[j] = result[j], result[i]
        else:
            random.shuffle(result)
        
        return ''.join(result)
    
    def generate_batch(self, count: int, length: int, secure: bool = False) -> List[str]:
        """
        Generate multiple random strings at once.
        
        Args:
            count: Number of strings to generate
            length: Length of each string
            secure: Use cryptographically secure random
            
        Returns:
            List of random strings
        """
        return [self.generate_random_string(length, secure) for _ in range(count)]

# Predefined generators for common use cases
class PresetGenerators:
    """Common preset generators for different use cases."""
    
    @staticmethod
    def hex_generator():
        """Generator for hexadecimal characters (0-9, A-F)"""
        return CustomRandomGenerator(
            digits='0123456789',
            lowercase='',
            uppercase='ABCDEF',
            special_chars=''
        )
    
    @staticmethod
    def alphanumeric_generator():
        """Generator for alphanumeric characters only"""
        return CustomRandomGenerator(special_chars='')
    
    @staticmethod
    def numeric_only():
        """Generator for numbers only"""
        return CustomRandomGenerator(
            lowercase='',
            uppercase='',
            special_chars=''
        )
    
    @staticmethod
    def vowels_only():
        """Generator using only vowels and digits"""
        return CustomRandomGenerator(
            lowercase='aeiou',
            uppercase='AEIOU',
            special_chars=''
        )
    
    @staticmethod
    def safe_characters():
        """Generator avoiding ambiguous characters (0, O, I, l, etc.)"""
        return CustomRandomGenerator(
            digits='123456789',
            lowercase='abcdefghijkmnopqrstuvwxyz',  # no 'l'
            uppercase='ABCDEFGHJKLMNPQRSTUVWXYZ',   # no 'I', 'O'
            special_chars=''
        )

# Example usage and demonstrations
def demonstrate_generators():
    """Demonstrate various generator configurations and usage."""
    
    print("=== Custom Random String Generation Examples ===\n")
    
    # 1. Basic usage with default character set
    print("1. Default generator (all digits and letters):")
    default_gen = CustomRandomGenerator()
    print(f"   Random string (length 12): {default_gen.generate_random_string(12)}")
    print(f"   Secure random (length 12): {default_gen.generate_random_string(12, secure=True)}")
    print()
    
    # 2. Custom subset example
    print("2. Custom subset (only digits 1-5 and vowels):")
    custom_gen = CustomRandomGenerator(
        digits='12345',
        lowercase='aeiou',
        uppercase='AEIOU',
        special_chars='!@#'
    )
    print(f"   Random string: {custom_gen.generate_random_string(10)}")
    print()
    
    # 3. Constrained generation
    print("3. Generation with constraints:")
    constrained = custom_gen.generate_with_constraints(
        length=12,
        min_digits=2,
        min_lowercase=2,
        min_uppercase=2,
        min_special=1
    )
    print(f"   Constrained string: {constrained}")
    print()
    
    # 4. Preset generators
    print("4. Preset generators:")
    print(f"   Hex string: {PresetGenerators.hex_generator().generate_random_string(8)}")
    print(f"   Numeric only: {PresetGenerators.numeric_only().generate_random_string(6)}")
    print(f"   Safe chars: {PresetGenerators.safe_characters().generate_random_string(10)}")
    print(f"   Vowels only: {PresetGenerators.vowels_only().generate_random_string(8)}")
    print()
    
    # 5. Batch generation
    print("5. Batch generation:")
    batch = default_gen.generate_batch(count=5, length=8)
    for i, item in enumerate(batch, 1):
        print(f"   Batch item {i}: {item}")
    print()
    
    # 6. Password-like generation
    print("6. Password generation example:")
    password_gen = CustomRandomGenerator(special_chars='!@#$%^&*')
    password = password_gen.generate_with_constraints(
        length=16,
        min_digits=2,
        min_lowercase=4,
        min_uppercase=4,
        min_special=2,
        secure=True
    )
    print(f"   Secure password: {password}")
    print()

if __name__ == "__main__":
    demonstrate_generators()

=== Custom Random String Generation Examples ===

1. Default generator (all digits and letters):
   Random string (length 12): R51TAFmLhrcw
   Secure random (length 12): 2k8r8aGGGmGX

2. Custom subset (only digits 1-5 and vowels):
   Random string: !IiaAa51o1

3. Generation with constraints:
   Constrained string: I3uOE#14aAE5

4. Preset generators:
   Hex string: wv0uCCtk
   Numeric only: BKfJ9W
   Safe chars: xQ3gZE6s1K
   Vowels only: Aauuuaai

5. Batch generation:
   Batch item 1: 8jtsmMSz
   Batch item 2: kqGvOERL
   Batch item 3: ytZ7nh7N
   Batch item 4: 0ndR42Cn
   Batch item 5: gdEU6cyg

6. Password generation example:
   Secure password: Il8VQg@Hyx1pJC*1

